# HOP Team Project - The Hindenburg Group
Tim Claytor

![Baloon](https://cdn.britannica.com/29/1229-004-0E10D8E6/Hindenburg-flames-Lakehurst-Naval-Air-Station-New-May-6-1937.jpg?s=750x300&q=85) 


___

* Library imports

In [5]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

## Tasks
* ~~We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.~~
* First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?
* Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume. 
    - First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).
    - Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?
* Finally, look for "communities" of providers in the Nashville/Davidson County CBSA. Make use of the Louvain community detection algorithm from Neo4j: https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/.

# Smaller Import Dataframes (non-chunky) 

In [ ]:
# Taxonomy table
taxonomy = pd.read_csv('../data/nucc_taxonomy_230.csv', encoding = 'unicode_escape')
taxonomy.info()
taxonomy.head()


In [69]:
# Zip Tract Table
all_zip = pd.read_excel('../data/ZIP_TRACT_122021.xlsx', index_col = None, header = 0, dtype={'zip': object})
all_zip.head()

,zip,tract,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00683,72023830102,SAN GERMAN,PR,0.000791,0.001116,0.000000,0.000800
1,00683,72125840700,SAN GERMAN,PR,0.186219,0.370536,0.381643,0.201179
2,00683,72125840400,SAN GERMAN,PR,0.300451,0.187500,0.115942,0.290308
3,00683,72125840600,SAN GERMAN,PR,0.095325,0.007812,0.000000,0.088184
4,00683,72121960300,SAN GERMAN,PR,0.042402,0.002232,0.019324,0.039435


In [102]:
# List to hold output of loop
nashville_tract = zip[(zip[usps_zip_pref_city] == 'NASHVILLE') & (zip[usps_zip_pref_state] == 'TN')]


NameError: name 'usps_zip_pref_city' is not defined

# Chunky Dataset Imports 

## HOP Teaming dataset

In [31]:
#Reading in DocGraph HOP Teaming data
hops_and_dreams = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
# List to hold output of loop
hop_chunks = []
# Loop for filtering criteria
for hop in hops_and_dreams:
    # transaction_count <= 50 & average_day_wait < 50
    hop = hop[(hop.transaction_count>= 50) & (hop.average_day_wait< 50)]
    # appending output to hop_chunks list
    hop_chunks.append(hop)


In [34]:
# Concatenating hop_chunks list to hop dataframe
hop = pd.concat(hop_chunks, ignore_index = True)
hop.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053
